In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
#Agile rules saved as a list
data_rules=['''Set a Clear Sprint Goal: Align the sprint goal with the product vision, incorporating user feedback and business priorities 

 

Sub-actions: 

Gather Input: Solicit ideas and input from all team members to ensure the sprint goal reflects diverse perspectives and expertise. 

Review Product Vision and Roadmap: Align the team on the product vision and how the sprint goal fits into the broader roadmap. 

Incorporate User Feedback: Analyze recent user feedback or survey data to ensure the sprint goal addresses current user needs or pain points. 

Finalize and Communicate Goal: Clearly articulate the finalized sprint goal to the team and ensure it is documented in a shared, accessible location ''','''Select Backlog Items: Collaboratively review and select backlog items that contribute directly to the sprint goal, ensuring a balanced workload. 

 

Sub-actions: 

Prioritize for Value and Urgency: Use a prioritization framework (e.g., MoSCoW, value vs. effort matrix) to assess items based on value and urgency. 

Ensure Alignment with Sprint Goal: Confirm that each selected backlog item directly contributes to achieving the sprint goal. 

Balance Workload: Assess the team’s capacity to ensure a realistic workload and avoid over commitment. 

Document Selection Rationale: Record the reasons for selecting each item for transparency and future reference. ''','''Task Breakdown and Estimation: Facilitate a task breakdown session, dividing complex items into manageable tasks. Implement an estimation exercise, such as Planning Poker, to gauge the effort required for each task, ensuring every team member's input is valued. 

 

Sub-actions: 

Prioritize for Value and Urgency: Use a prioritization framework (e.g., MoSCoW, value vs. effort matrix) to assess items based on value and urgency. 

Ensure Alignment with Sprint Goal: Confirm that each selected backlog item directly contributes to achieving the sprint goal. 

Balance Workload: Assess the team’s capacity to ensure a realistic workload and avoid over commitment. 

Document Selection Rationale: Record the reasons for selecting each item for transparency and future reference.''','''Encourage Participation: Use techniques to ensure all team members contribute their perspectives and expertise, fostering a sense of ownership and commitment. 

 

Sub-actions: 

Create an Inclusive Environment: Ensure that the meeting setup encourages participation from all, including remote team members. 

Rotate Facilitation: Allow different team members to facilitate parts of the sprint planning to increase engagement. 

Use Engagement Techniques: Implement techniques such as silent writing for idea generation and dot voting for prioritization to ensure diverse input. 

Acknowledge Contributions: Recognize and appreciate the input from all team members to foster a culture of contribution. ''','''Address Challenges and Blockers: Proactively identify potential challenges or blockers that could impact sprint execution and discuss strategies to mitigate these risks. 

 

Sub-actions: 

Identify Risks Early: Have team members identify potential risks and blockers at the beginning of planning. 

Strategy Development: For each identified challenge, brainstorm potential mitigation strategies as a team. 

Assign Owners to Challenges: Designate responsibility for monitoring and addressing each risk to specific team members or sub-teams. 

Create a Risk Tracker: Use a shared document or tool to track identified risks, their status, and mitigation actions over the sprint. 

Regular Review: Incorporate a brief review of risks and blockers in daily stand-ups or dedicated sessions to ensure proactive management.''']

In [4]:
from openai import AzureOpenAI
openai_client=AzureOpenAI(
  api_key =  os.getenv("api_key"),
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("azure_endpoint"))

#Embedding model for Azure AI search
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai_client.embeddings.create(input = [text], model=model).data[0].embedding

In [5]:
#Structuring data for creating the search index
data=[]
counter=0
for text in data_rules:

    d={}
    d['id'] = str(counter)
    d['rules']=text.split(":")[0]
    d['description']=text
    d['Embedding']=get_embedding(text)
    counter=counter+1
    data.append(d)

In [6]:

print(data[2])

{'id': '2', 'rules': 'Task Breakdown and Estimation', 'description': "Task Breakdown and Estimation: Facilitate a task breakdown session, dividing complex items into manageable tasks. Implement an estimation exercise, such as Planning Poker, to gauge the effort required for each task, ensuring every team member's input is valued. \n\n \n\nSub-actions: \n\nPrioritize for Value and Urgency: Use a prioritization framework (e.g., MoSCoW, value vs. effort matrix) to assess items based on value and urgency. \n\nEnsure Alignment with Sprint Goal: Confirm that each selected backlog item directly contributes to achieving the sprint goal. \n\nBalance Workload: Assess the team’s capacity to ensure a realistic workload and avoid over commitment. \n\nDocument Selection Rationale: Record the reasons for selecting each item for transparency and future reference.", 'Embedding': [-0.014830926433205605, -0.0012546881334856153, 0.02563319355249405, -0.018763989210128784, -0.023147715255618095, 0.00816656

In [7]:
len(data[0]['Embedding'])

1536

In [8]:
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import (  
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticPrioritizedFields,
    SemanticField,  
    SearchField,  
    SemanticSearch,
    VectorSearch,  
    HnswAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticField,  
    SearchField,  
    VectorSearch,  
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)  
 

In [9]:
#Azure AI Search index client
service_endpoint =os.getenv("ai_search_endpoint")
# creating an index and naming for future reference/searching and will reflect in azure portal
index_name = "loliprop"
key = os.getenv("search_index_key")
credential = AzureKeyCredential(key)
index_client=SearchIndexClient(endpoint=service_endpoint, credential=credential)
# id is number and rules is also name of the rule so we make a simple field there
# searchable field and search field are the ones relevant to our use case
# Embedding is the field that will be used for vector search
# searchable focuses on frequency of repetitions whereas search field focuses more on semantic meaning
# HNSW is a algorithm on how model performs its search, adv - less computation and results similar to industry standard models
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, 
                key=True, sortable=True, 
                filterable=True, facetable=True),
    SimpleField(name="rules", type=SearchFieldDataType.String, 
                key=False, 
                filterable=True, facetable=True),
    SearchableField(name="description", type=SearchFieldDataType.String,searchable=True),
    SearchField(name="Embedding", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, 
                vector_search_profile_name="myHnswProfile")
]

/Users/pulkitgarg/mypop/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
index_client

In [11]:
# Configure the vector search configuration  
# use profile names to switch models currently using MyHnsw
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm_configuration_name="myExhaustiveKnn",
        )
    ]
)

In [12]:
#semantic Search config
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        
        content_fields=[SemanticField(field_name="description")]
    )
)
 

In [13]:
# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, 
                    semantic_search=semantic_search)
result = index_client.create_or_update_index(index)

print(f' {result.name} created')

 loliprop created


In [14]:
from azure.search.documents import SearchClient
#upload data to search index
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(data)
print(f"Uploaded {len(data)} documents")
 

Uploaded 5 documents


In [19]:
# We have 3 types of search and only required to run one
#FunctionforRAG
# Vector Search

def RAGoutput(query):
    query_vector =get_embedding(query, model='text-embedding-ada-002')

    vector_query = VectorizedQuery(vector=query_vector, 
                                fields="Embedding")
    results = search_client.search(  
        # freq search hence the field is zero
        # if you want to do hybrid search, pass query(not the vector) to search_text field
        # search_text = query
        search_text=None,
        vector_queries=[vector_query],  
        select=["id","rules","description"],
        )
    for result in results:
        output=result["description"]
        break
    return output

In [16]:
#FunctionforRAG
# Hybrid Search

def RAGoutput(query):
    query_vector =get_embedding(query, model='text-embedding-ada-002')

    vector_query = VectorizedQuery(vector=query_vector, 
                                fields="Embedding")
    results = search_client.search(  
        # freq search hence the field is zero
        # if you want to do hybrid search, pass query(not the vector) to search_text field
        search_text=query,
        vector_queries=[vector_query],  
        select=["id","rules","description"],
        )
    for result in results:
        output=result["description"]
        break
    return output

In [17]:
#FunctionforRAG
# Frequency Search
# we do not need to convert queries into embeddings for frequency search

def RAGoutput(query):
    query_vector =get_embedding(query, model='text-embedding-ada-002')

    vector_query = VectorizedQuery(vector=query_vector, 
                                fields="Embedding")
    results = search_client.search(  
        # freq search hence the field is zero
        # if you want to do hybrid search, pass query(not the vector) to search_text field
        search_text=query,
        vector_queries=None,  
        select=["id","rules","description"],
        )
    for result in results:
        output=result["description"]
        break
    return output

In [22]:
#chat conversation split and saved as a list
conversation='''Alice: Good morning, everyone! Thank you for joining today's Sprint Planning meeting for the NextGen Shopping App. As we begin, our main objectives are to set our sprint goal, select the right backlog items, and estimate our tasks accurately. Let's make sure we're all contributing to the discussion, sharing insights, and making decisions together. Bob, could you start us off by revisiting our product vision and highlighting how this sprint fits into that? 

 

Bob: Sure, Alice. As we all know, our vision for the NextGen Shopping App is to create a user-centric shopping experience that's not just about buying but about discovering and enjoying the process. Each sprint brings us closer to this goal by incrementally improving our app. This sprint, I propose we focus on enhancing user satisfaction through speed and ease of use, based on recent feedback. 

 

Carla: I've been analyzing the user feedback, and it's clear that our checkout process needs to be faster and more intuitive. Our users want a seamless experience, and right now, we're not quite there. 

 

Dave: I agree with Carla. From a backend perspective, there are several optimizations we can make to improve data processing and reduce load times. It will directly contribute to making the app faster. 

 

Alice: Excellent points. Let's align our sprint goal to focus on optimizing the checkout process and improving app speed. This aligns with our product vision by directly enhancing user satisfaction. Does everyone agree? 

 

General nods and agreements 

 

Emily: From the frontend side, I already have some ideas on how to streamline the checkout UI. We can simplify the steps involved and make the interface more responsive. 

 

Frank: And I'll make sure to focus on testing these changes extensively. We need to ensure that any improvements don't introduce new bugs, especially in the checkout flow. 

 

Alice: Great! It sounds like we have a solid direction for this sprint. Our goal will be to enhance the checkout process and overall app speed, aligning with user feedback and our long-term vision. Let's move on to selecting the specific backlog items that will help us achieve this goal. Carla, could you lead that discussion? 

Chapter 2: Selecting Backlog Items and Balancing Workload 

Carla: Following Alice's lead, let's dive into selecting our backlog items. Based on our sprint goal of improving the checkout process and app speed, I've identified a list of potential enhancements and bug fixes. However, it's crucial we prioritize items that offer the most value to our users while considering our capacity. 

 

Bob: Carla, can you walk us through your top picks for this sprint? 

 

Carla: Sure, Bob. First, we have the redesign of the checkout UI to make it more intuitive. Next, there are backend optimizations for faster data processing, particularly during the checkout phase. And we also have a task to address a frequently reported bug that slows down the app when loading product images. 

 

Dave: The backend optimizations are critical. I've looked into the current processing times, and there's definitely room for improvement. But we'll need to ensure we're not overloading the database with requests. 

 

Emily: I'm on board with the checkout UI redesign. It aligns perfectly with my ideas for enhancing the frontend. We can introduce a progress bar and reduce the number of steps required to complete a purchase. 

 

Frank: Testing these changes will be key. I propose we allocate extra time for performance testing, especially after the backend optimizations and UI redesign. 

 

Alice: Excellent. It seems we have a consensus on prioritizing these items. Now, let's talk about workload balance. We need to ensure that the tasks are evenly distributed and that we're not overcommitting. How does our capacity look for this sprint? 

 

Bob: I've reviewed our capacity, and considering the complexity of the backend optimizations and UI redesign, we'll need to be mindful of our limits. It might mean postponing less critical items to the next sprint. 

 

Carla: I agree with Bob. Let's focus on what's most impactful for our users and ensure we have a balanced approach. We can re-evaluate our progress mid-sprint and adjust if needed. 

 

Dave: I'll need to pair up with Emily on some of the backend changes to ensure they align well with the frontend updates. That should help us stay efficient. 

 

Emily: Sounds good, Dave. We'll also need to collaborate closely with Frank to plan out the testing phases. 

 

Alice: Alright, team, let's document our selected backlog items and the rationale behind each choice. This documentation will not only keep us aligned but also serve as a reference for future sprints. Now, moving on to breaking down these tasks and estimating our effort. This is where we need everyone's input to ensure our estimations are accurate and reflect our capabilities. Emily, could you lead us into the task breakdown and estimation process? 

 

Chapter 3: Task Breakdown and Effort Estimation 

Emily: Thanks, Alice. Let's start with the task breakdown. We've identified the key areas we're focusing on this sprint: the checkout UI redesign, backend optimizations, and the bug fix for image loading. Breaking these down into smaller, manageable tasks will help us understand the effort required and ensure we cover all necessary work without overcommitting. 

 

First, the checkout UI redesign: I propose dividing it into three main tasks - wireframing the new design, implementing the changes, and then user testing. Wireframing involves drafting the new layout and getting team feedback. Implementation is the coding part, where we actually build the new design. User testing involves having a small group of actual users try out the redesigned checkout process and provide feedback. 

 

For backend optimizations: We need to analyze current performance bottlenecks, refactor the code for efficiency, and then conduct stress testing. Analyzing bottlenecks will help us identify the exact points where the app slows down during checkout. Refactoring is where we clean up and optimize the existing code to eliminate these bottlenecks. Stress testing will ensure the backend can handle peak loads, especially during the checkout process. 

 

Regarding the bug fix for image loading: This can be broken down into diagnosing the issue, fixing the bug, and then validating the fix. Diagnosing requires us to replicate the bug and find out why it's happening. Fixing it is the coding work to eliminate the bug. Validating ensures that the fix works under all expected conditions and doesn't introduce new issues. 

 

Dave: That's a solid breakdown, Emily. For the backend optimizations, I'll take the lead on analyzing performance bottlenecks. I'll need some help from Carla with identifying which processes are most critical to user satisfaction during checkout. 

 

Carla: Happy to collaborate, Dave. I'll gather user feedback and analytics to pinpoint where users are experiencing slowdowns. 

 

Frank: I'll focus on the testing aspects. For the UI redesign, I'll set up user testing sessions. And for backend optimizations and the bug fix, I'll conduct the stress testing and validation to ensure our changes are effective and stable. 

 

Alice: Great, it sounds like we have a clear understanding of the tasks ahead and who's responsible for each part. Now, let's move on to estimating these tasks using Planning Poker. This will help us ensure that our estimates reflect the team's diverse perspectives and expertise. 

 

Bob: Before we start, remember that our goal here is not to nail down the exact number of hours each task will take but to understand the relative complexity and effort involved. This helps us allocate our time and resources effectively for the sprint. 

 

Alice: Exactly, Bob. Let's start with the checkout UI redesign wireframing. How complex do we think this task is compared to our baseline tasks? 

 

The team engages in a Planning Poker session, holding up cards with their estimates. After revealing their estimates, there's a discussion on the reasoning behind the different numbers, leading to a consensus on the effort required. 

 

Alice: Looks like we have agreement on a moderate complexity for the wireframing. Now, implementing the redesign. What are our thoughts? 

 

The process repeats, with team members debating the complexity of implementing the redesign, considering the need for collaboration between frontend and backend teams to ensure the changes are supported. 

 

Alice: And finally, the backend optimizations. Let's estimate the combined effort for analyzing bottlenecks, refactoring, and stress testing. 

 

Again, the team discusses and reaches a consensus, this time recognizing the task as one of the more complex efforts due to the technical challenges involved. 

 

Alice: Thank you, everyone, for your thoughtful participation. We now have a clear picture of the effort required for each major task in our sprint. This will be invaluable as we plan our work and manage our time. Let's document these estimations and ensure everyone has access to this breakdown. 

 

Bob: With our tasks clearly defined and estimated, we're in a good position to start this sprint strong. Let's commit to regular check-ins to ensure we're on track and adjust as needed. It's going to be a busy sprint, but I'm confident we can make significant improvements to the NextGen Shopping App. 

 

Alice: Absolutely, Bob. This detailed planning sets us up for success. Let's keep communication open, support each other, and push towards achieving our sprint goal. Thanks, everyone, for your hard work today. Let's make this sprint a win for our users and for us! 

 

Chapter 4: Encouraging Full Team Participation and Addressing Challenges 

As the Sprint Planning meeting continued, Alice shifted the focus towards ensuring full team participation and preemptively addressing potential challenges and blockers that could arise during the sprint. 

 

Alice: Now that we have our tasks defined and estimated, it's crucial that we ensure everyone has a voice in this process. We're a team, and every perspective is valuable. Let's discuss any concerns about the upcoming sprint, and together, we'll brainstorm solutions or mitigations. 

 

Bob: I want to make sure we're not overlooking the integration between the new checkout UI and the backend. It's a critical piece of this sprint, and misalignment could cause delays. 

 

Carla: That's a valid concern, Bob. Perhaps we could schedule a mid-sprint check-in specifically focused on this integration. It would give both the frontend and backend teams a clear deadline for their initial work, ensuring we have time to adjust if needed. 

 

Dave: I'm on board with that. Also, from a technical standpoint, we need to be prepared for any unexpected challenges with the backend optimizations. They're complex, and while we've estimated the effort, there's always a risk of encountering something unforeseen. 

 

Emily: On the frontend side, I'll make sure to keep the communication lines open with Dave and his team. If we encounter any issues with the UI that could impact the backend, or vice versa, we'll raise them immediately. 

 

Frank: From a QA perspective, my challenge will be ensuring we have adequate coverage for both the new features and regression testing. I might need support in prioritizing the test cases. 

 

Alice: These are all important points. Let's address them systematically. For the integration check-in, let's set a date now. How does next Wednesday sound to everyone? 

 

The team agrees on the date. 

 

Alice: Great, it's scheduled. Dave, regarding potential backend surprises, let's allocate some buffer time in our sprint for unexpected work. We won't assign it to specific tasks now, but it'll be there if we need it. 

 

Dave: Sounds like a good safety net, Alice. Appreciate it. 

 

Alice: Emily, your proactive communication will be key. Let's also schedule brief daily check-ins between frontend and backend teams, just a quick stand-up to ensure everything's aligned. 

 

Emily: Will do, Alice. Those stand-ups will help catch any issues early. 

 

Alice: And Frank, for the testing coverage, let's have a session tomorrow to go through the test cases and prioritize them together. We can look at automating some of them to save time. 

 

Frank: That would be really helpful, Alice. Thanks. 

 

Alice: Before we wrap up, does anyone else have any concerns or suggestions? This is the time to bring them up. 

 

The team takes a moment, but no further issues are raised. 

 

Alice: Alright, then. This has been a highly productive discussion. We've set a solid foundation for our sprint, addressing not only our goals and tasks but also how we'll work together to overcome challenges. Remember, flexibility and communication are key. Let's keep the momentum going and support each other throughout the sprint. 

 

Bob: I just want to say, I'm really impressed with how we've tackled this planning session. It's clear we're all committed to making this sprint successful, and I'm confident we'll overcome any challenges that come our way. 

 

Alice: Thanks, Bob. And thank you to each one of you for your contributions today. Let's go into this sprint with a positive attitude and a commitment to making the NextGen Shopping App even better for our users. 

Chapter 5: Strategy Development and Risk Management 

With the tasks defined, estimated, and the team aligned on addressing potential challenges, Alice directed the meeting's focus toward developing strategies for the identified risks and ensuring a robust plan for risk management throughout the sprint. 

 

Alice: Having laid a strong foundation for our sprint, it's time we shift our focus to developing strategies for the risks we've identified. A proactive approach here will help us navigate the sprint more smoothly and avoid being caught off guard by any issues. Let’s start by listing the key risks we foresee and brainstorming strategies to mitigate them. 

 

Bob: One risk that stands out to me is the potential for scope creep, especially with the checkout UI redesign. It's an area where we could easily find ourselves adding 'just one more feature' and impacting our sprint timeline. 

 

Carla: To mitigate that, we could define clear acceptance criteria for what we consider 'complete' for the sprint's purpose. Anything beyond that gets logged as a future enhancement but doesn't derail our current sprint. 

 

Alice: That's a great approach, Carla. Having strict acceptance criteria will help maintain our focus. Dave, what do you see as a potential risk from the backend side? 

 

Dave: The main risk I see is underestimating the complexity of the optimizations we're planning. If we run into unexpected technical debt, it could slow us down significantly. 

 

Emily: Maybe we could allocate specific time slots for code review and refactoring as part of our sprint activities. This way, we can tackle technical debt more systematically rather than letting it surprise us. 

 

Alice: Allocating time for dealing with technical debt is a smart strategy, Emily. It ensures we're not just firefighting but also improving our codebase's health. Frank, from a QA perspective, what risks do you anticipate? 

 

Frank: The biggest risk I see is not having enough time for thorough testing, especially with the new features and optimizations. We might miss critical bugs. 

 

Alice: To counter that, we could use a risk-based testing approach. We prioritize testing based on the potential impact of bugs in different areas of the app. This way, we ensure we're focusing our efforts where they're most needed. 

 

Bob: I think another risk could be failing to meet user expectations with the checkout UI redesign. Even if we improve it technically, it might not align with what users find intuitive or helpful. 

 

Carla: Conducting user testing sessions early in the sprint could help mitigate this. We get feedback on our prototypes before we're too far down the development path, allowing us to pivot if necessary. 

 

Alice: User testing is indeed a powerful tool for ensuring we're on the right track. Now, let's talk about how we'll manage these risks throughout the sprint. We need a systematic approach to monitor these risks and respond if any of them begin to materialize. 

 

Dave: How about setting up a risk register? We can list all identified risks along with their mitigation strategies, owners, and current status. This could be a living document we review and update regularly. 

 

Alice: A risk register is a fantastic idea, Dave. It will give us a clear overview of our risk landscape and help ensure nothing falls through the cracks. We can incorporate it into our daily stand-ups or have a dedicated time each week to review it as a team. 

 

Emily: I think regular check-ins on risks will keep us all aware and proactive. It’s easy to get caught up in the day-to-day work and forget to look at the bigger picture. 

 

Alice: Absolutely, Emily. Regularly revisiting our risk register will ensure we're not only aware of potential issues but also actively managing them. This kind of proactive risk management is what can make the difference between a successful sprint and a challenging one. 

 

Bob: This all sounds like a solid plan. With clear strategies for our known risks and a system in place to manage them, I feel confident we're prepared for a productive sprint. 

 

Alice: I'm glad to hear that, Bob. And thank you, everyone, for contributing to this important discussion. Risk management is a critical part of our sprint planning, and by addressing it thoughtfully, we're setting ourselves up for success. Let's keep these strategies in mind and support each other in monitoring and managing risks as we move forward. 

 

Frank: It’s reassuring to have these strategies and the risk register in place. Knowing we have a plan to handle potential issues makes me feel much more confident about tackling this sprint's challenges. 

 

Alice: With that, we’ve covered our main objectives for today's Sprint Planning meeting. We’ve set our sprint goal, selected our backlog items, broken down tasks and estimated effort, encouraged full participation, and developed strategies for managing risks. It’s been an incredibly productive session, and I’m excited to see what we’ll accomplish together in this sprint. Let’s make it a great sprint! 

Chapter 6: Finalizing Sprint Commitments and Preparing for Execution 

With strategies for managing identified risks firmly in place, the team moved on to finalize their commitments for the sprint and prepare for the execution phase. Alice facilitated this crucial part of the meeting with an emphasis on ensuring that every team member felt confident and clear about their responsibilities and the team’s collective goals. 

 

Alice: We’ve made significant progress today, from setting our sprint goal to devising a comprehensive risk management plan. Now, let’s finalize our sprint commitments. This is where we commit as a team to the work we plan to complete by the end of this sprint. It’s important that we all agree and are on the same page with what we’re setting out to achieve. 

 

Bob: Before we make our final commitments, let’s quickly recap the major tasks we’ve agreed upon and ensure everyone understands their roles and the expectations. 

 

Carla: From the design perspective, we're committed to finalizing the wireframes early in the sprint, collaborating with Emily on implementing the UI changes, and preparing for user testing sessions. 

 

Dave: On the backend side, I’ll lead the performance analysis and begin the optimization work. I’ve also allocated time to support Emily with any backend changes needed for the new UI. 

 

Emily: For the frontend, I’m focusing on implementing the new checkout process UI, based on Carla’s wireframes and user feedback. Plus, I'll be working closely with Dave to ensure smooth integration. 

 

Frank: And I'll be preparing our testing environment early on, focusing on setting up automated tests for the new features and conducting thorough performance and regression testing throughout the sprint. 

 

Alice: Thanks for those summaries. It sounds like we’re all clear on our main tasks and responsibilities. Now, let’s talk about our definition of 'Done.' It’s crucial we all agree on what it means for a task to be considered complete, ensuring there are no misunderstandings and that we maintain high-quality standards. 

 

Bob: Absolutely, Alice. For me, a task is 'Done' when it’s been developed, tested, reviewed by peers, and meets all the acceptance criteria we’ve defined. 

 

Carla: And from a design perspective, it also means it has passed user testing with positive feedback or identified issues have been addressed. 

 

Alice: Excellent points. Having a shared definition of 'Done' is critical for our team’s alignment and success. Now, with our tasks, roles, and definition of 'Done' clear, I’d like us to officially make our sprint commitments. Are we all comfortable and confident with the plan we’ve laid out? 

 

The team members look around the table at one another, nodding and expressing their agreement and readiness to proceed. 

 

Alice: I’m seeing nods all around, which is great. This commitment is a promise to ourselves and each other that we’ll do our best to achieve our sprint goal. Let’s support each other and communicate openly throughout the sprint, especially if any issues arise or if anyone needs help. 

 

Dave: I think it’s also important to mention that while we’re committed, we should remain flexible. If we encounter unexpected challenges or discover new information that affects our work, we should feel empowered to bring it up and adjust our plans accordingly. 

 

Alice: Absolutely, Dave. Agility is all about adapting to change, and we should always be prepared to reassess our priorities and adapt our plans to ensure we’re delivering the most value to our users. 

 

Bob: I want to thank everyone for their contributions today. This planning session has set a strong foundation for our sprint. Let’s keep the momentum going and make this sprint a success. 

 

Alice: As we wrap up, let’s confirm our first daily stand-up meeting for tomorrow morning. It’ll be a quick 15-minute check-in to share our progress and any blockers. Remember, the goal of these stand-ups is to ensure we’re all aligned and to provide support where needed. 

 

Emily: Looking forward to getting started and seeing all our plans come to life! 

 

Alice: Thank you, everyone, for your hard work and dedication today. This planning session has been incredibly productive, and I’m confident we’re on the right path to achieving our sprint goal. Let’s bring our best to this sprint and make a positive impact on the NextGen Shopping App. See you all at the daily stand-up tomorrow! 

 

Chapter 7: Reflection and Anticipation: Closing the Sprint Planning Meeting 

As the Sprint Planning meeting reached its final phase, Alice encapsulated the session with a moment of reflection and forward-looking anticipation. The team had meticulously navigated through setting goals, selecting backlog items, breaking down tasks, estimating efforts, encouraging full participation, and finalizing commitments. Now was the moment to ensure everything was in place for a successful sprint execution and to set the tone for the upcoming work. 

 

Alice: As we close today's meeting, I want to take a moment for us to reflect on the journey we've embarked on together this morning. We've set ambitious goals and laid out a clear path to achieve them. Before we adjourn, I'd like each of us to share our thoughts on the planning process and express any final concerns or ideas. This is also a time to voice how we feel going into the sprint—any excitement, apprehensions, or questions. 

 

Bob: I'll start. I'm really impressed with the level of detail and thought everyone has put into today's planning. It's clear we're all deeply invested in the success of the NextGen Shopping App. I'm particularly excited about the checkout UI redesign. I believe it will make a significant difference in our user experience. My only apprehension is ensuring we stay on track with our timeline, but given our planning today, I'm optimistic. 

 

Carla: Reflecting on today, I'm grateful for the collaborative spirit of this team. The planning poker session was a highlight for me—it really showcased how each of us brings a unique perspective to the table. I'm excited to see the user reactions to the redesign. My main focus will be ensuring our designs are not only beautiful but also intuitive for our users. 

 

Dave: From a backend perspective, today’s meeting reinforced the importance of our behind-the-scenes work to the overall user experience. The tasks ahead are challenging but also exciting. I feel we have a solid plan to tackle the optimizations needed. My only concern is encountering unforeseen technical debt, but I’m confident in our team’s ability to adapt and address issues as they arise. 

 

Emily: I'm really looking forward to implementing the new UI designs and working closely with the backend team to ensure everything integrates smoothly. It's going to be a lot of work, but the planning we've done today makes me feel prepared. I’m particularly keen on making the app faster and more responsive. Seeing our plans on paper (or screen) turning into real, impactful changes for our users is always rewarding. 

 

Frank: As the QA lead, I appreciate the thorough discussion around testing and risk management today. It’s clear we’re not just focusing on new features but also on the quality and stability of the app. I’m excited to set up our testing frameworks for the new features and to ensure that our app continues to meet the high standards our users expect. My focus will be on keeping a close eye on the integration points and ensuring they are rigorously tested. 

 

Alice: Thank you, everyone, for your reflections. It's clear that we're starting this sprint with a strong sense of purpose and commitment. I want to echo the sentiments shared and add my own excitement for what we're about to achieve. The road ahead will require hard work, dedication, and flexibility. Remember, the strength of our team lies not just in our individual abilities but in our collective effort and support for each other. 

 

Alice continued: As we move forward, let’s keep the lines of communication open. Our daily stand-ups will be a key time to share updates and address any obstacles. Beyond that, I encourage you to reach out to your teammates whenever you need assistance or have insights to share. Together, we can make this sprint a milestone in the NextGen Shopping App’s journey. 

 

Alice: Finally, I want to remind us all of the importance of maintaining a healthy work-life balance. The enthusiasm and commitment we’ve shown today are phenomenal, but let’s also ensure we’re taking care of ourselves. A well-rested and happy team is a productive team. 

 

Alice: As we conclude, I want to say thank you. Thank you for your hard work, your insights, and your dedication not just to the project, but to each other. Let’s go into this sprint with confidence and make it a success. Here’s to a fantastic sprint ahead! '''.split('\n')

In [26]:
print(conversation)

["Bob: Sure, Alice. As we all know, our vision for the NextGen Shopping App is to create a user-centric shopping experience that's not just about buying but about discovering and enjoying the process. Each sprint brings us closer to this goal by incrementally improving our app. This sprint, I propose we focus on enhancing user satisfaction through speed and ease of use, based on recent feedback. ", '', ' ', '']


In [27]:
conversation = [s.strip() for s in conversation if s.strip()]
 

In [30]:
print(conversation[2])

Carla: I've been analyzing the user feedback, and it's clear that our checkout process needs to be faster and more intuitive. Our users want a seamless experience, and right now, we're not quite there.


In [31]:
#Structure conversation user wise


user_dictionary= {}
for user in conversation:
    username=user.split(":")[0]
    # print(user.split(":")[1:])
    if username in user_dictionary:
        userinfo="".join(user.split(":")[1:]) #separate message from user name
        user_dictionary[username].append(userinfo)
    else:
        if(len(username.split())==1):  #Check if the user name is a single name
            userinfo="".join(user.split(":")[1:]) #separate message from user name
            user_dictionary[username]=[userinfo]     
    
 

In [32]:
for key in user_dictionary.keys():
    print(key)

Alice
Bob
Carla
Dave
Emily
Frank
